In [54]:
!pip install albumentations==0.4.6
!pip install ensemble-boxes

In [55]:
import pathlib
from google.colab import drive
import tqdm
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import cv2

import torch
import torchvision
from torchvision import transforms
import os
import numpy as np
import torch
from PIL import Image

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
# # model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)

In [57]:
# model

In [58]:
# from albumentations import RandomCrop, HorizontalFlip, VerticalFlip, RandomBrightnessContrast
import albumentations as A

In [59]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [60]:
# device = torch.device('cpu')

In [61]:
device

device(type='cuda')

In [69]:
root_path = pathlib.Path('/content/drive/MyDrive/anom_detection/task')

In [70]:
class SteelDataset(torch.utils.data.Dataset):
    def __init__(self, root):
        self.root = pathlib.Path(root)
        
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted((self.root/'images').iterdir()))
        self.boxes = list(sorted((self.root/'anno').iterdir()))

    def __getitem__(self, idx):
        # load images and masks
        img_path = self.root/'images'/self.imgs[idx]
        boxes_path = self.root/'anno'/self.boxes[idx]
        with open(boxes_path, 'r', encoding='utf-8') as g:
            anno_file = BeautifulSoup(g.read(), 'lxml-xml')
        img = cv2.imread(str(img_path))

        # img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img /= 255.0
        
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        # mask = Image.open(mask_path)
        # convert the PIL Image into a numpy array
        # mask = np.array(mask)
        # instances are encoded as different colors
        # obj_ids = np.unique(mask)
        # first id is the background, so remove it
        # obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        # masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        bboxes = []
        for tags in anno_file.find_all('object'):
            xmin = int(tags.find('bndbox').find('xmin').get_text())
            ymin = int(tags.find('bndbox').find('ymin').get_text())
            xmax = int(tags.find('bndbox').find('xmax').get_text())
            ymax = int(tags.find('bndbox').find('ymax').get_text())
            bboxes.append([xmin, ymin, xmax, ymax]) 


        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(bboxes, dtype=torch.float32)
        # print(boxes)
        # print(boxes)
        # there is only one class
        num_objs = 1
        # labels = torch.ones((len(self.imgs)), dtype=torch.float32)
        labels = torch.ones((boxes.shape[0],), dtype=torch.int64)
        # print(len(boxes))
        # print(len(labels))
        # print(len(img))
        # masks = torch.as_tensor(masks, dtype=torch.uint8)

        # image_id = torch.tensor([idx])
        # area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        # iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        # target["masks"] = masks
        # target["image_id"] = image_id
        # target["area"] = area
        # target["iscrowd"] = iscrowd


          
            # print('fdsfsdf')
            # if len(sample['bboxes']) > 0:
            #         image = sample['image']
            #         target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes']))))
                     
            # img = self.transforms(image=img)["image"]

        # boxes = torch.stack(tuple(map(torch.tensor, target['boxes'])))
        # data_dict = {'image' : img, 'boxes' : boxes}
        # return data_dict
        # print(img.shape)
        # print(len(boxes))
        return img, target

    def __len__(self):
        return len(self.imgs)

In [71]:
class AbstractDataset(torch.utils.data.Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
        
    def __getitem__(self, index):
        img, target = self.subset[index]
        if self.transform:
            sample = self.transform(**{
                    'image': img,
                    'bboxes': target['boxes'],
                    'labels': target['labels']
                      })
            img = sample['image']
            target['boxes'] = torch.as_tensor(sample['bboxes'], dtype=torch.float32)            
            target['labels'] = torch.as_tensor(sample['labels'], dtype=torch.int64)
           
        return img, target
        
    def __len__(self):
        return len(self.subset)

In [72]:
import torch
import torch.nn as nn
import torch.optim as optim 
import torchvision
from torchvision import models
from torch.utils.data import DataLoader, Dataset
import torch.utils.data as utils
from torchvision import transforms
import torch.nn.functional as F
import time

In [73]:
# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True, trainable_backbone_layers=3)

In [74]:
from albumentations.pytorch import ToTensorV2

In [75]:
# Randomcrop, Hflip, Vflip, RandomBrightnessContrast

# trans = transforms.Compose([
#             transforms.ToTensor(),      
#             transforms.Resize((224, 224)),
#             transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                      std=[0.229, 0.224, 0.225])                    
#         ])
def get_train_transforms():
    return A.Compose([
                      # A.RandomCrop(height=150, width=150, p=0.5),
                      A.RandomSizedBBoxSafeCrop(height=150, width=150, p=0.5),
                A.RandomBrightnessContrast(brightness_limit=0.2, 
                                           contrast_limit=0.2, p=0.9),
            A.HorizontalFlip(p=0.3),
            A.VerticalFlip(p=0.3),
            A.Resize(224, 224, interpolation=1),
            # A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2(p=1.0)
        ], 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        ))
    
def get_test_transforms():
    return A.Compose([
            A.Resize(224, 224, interpolation=1),
            # A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2(p=1.0)
        ], 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        ))

In [76]:
dataset = SteelDataset(root_path)
train_set, test_set = torch.utils.data.random_split(dataset, [int(len(dataset)*0.8), int(len(dataset)*0.2)])

In [77]:
train_data = AbstractDataset(
    train_set, transform=get_train_transforms()
)
test_data = AbstractDataset(
    test_set, 
    transform=get_test_transforms()
)

In [ ]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(params, lr=1e-5, weight_decay=1e-3)
lr_scheduler = None

In [ ]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [ ]:
def collate_fn(batch):
    """
    Since each image may have a different number of objects, we need a collate function (to be passed to the DataLoader).
    This describes how to combine these tensors of different sizes. We use lists.
    Note: this need not be defined in this Class, can be standalone.
    :param batch: an iterable of N sets from __getitem__()
    :return: a tensor of images, lists of varying-size tensors of bounding boxes, labels, and difficulties
    """

    # return images, target
    return tuple(zip(*batch))

In [ ]:
# from itertools import zip_longest

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True, 
                                               num_workers=2, 
                                               collate_fn=collate_fn, 
                                               drop_last=True
                                               )

test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=8, shuffle=True, # shuffle for test
                                               num_workers=2, 
                                              collate_fn=collate_fn,
                                              drop_last=True
                                              )

In [ ]:
num_epochs = 8

In [ ]:
loss_hist_train = Averager()
loss_hist_test = Averager()
# itr = 1

def train_epoch(images, targets):
    model.train()
    images = torch.stack(images)
    images = images.to(device).float()
    # targets = {}
    # targets['boxes'] = boxes
    # targets['labels'] = labels
    # print(targets)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

    # print(targets)
    loss_dict = model(images, targets)
    # print(loss_dict)
    losses = sum(loss for loss in loss_dict.values())
    loss_value = losses.item()

    loss_hist_train.send(loss_value)

    optimizer.zero_grad()
    losses.backward()
    optimizer.step()

    if lr_scheduler is not None:
        lr_scheduler.step()

def test_epoch(images, targets):
    model.eval()
    summary_loss = Averager()
    images = torch.stack(images)
    images = images.to(device).float()
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

    loss_dict = model(images, targets)
    # print(loss_dict)
    # losses = sum(loss for loss in loss_dict.values())
    loss_value = losses.item()

    loss_hist_test.send(loss_value)


for epoch in range(num_epochs):
    loss_hist_train.reset()
    loss_hist_test.reset()
    
    for images, targets in train_dataloader:
        train_epoch(images, targets)
        # print(images[0])
        # images = list(image.to(device) for image in images)
    # for images, targets in test_dataloader:
    #     with torch.no_grad():
    #         test_epoch(images, targets)


        # if itr % 50 == 0:
        #     print(f"Iteration #{itr} loss: {loss_value}")

        # itr += 1
    
    # update the learning rate


    print(f"Epoch #{epoch} train loss: {loss_hist_train.value}")
    # print(f"Epoch #{epoch} test loss: {loss_hist_test.value}")   

In [109]:
# def get_val_transforms():
#     return A.Compose([
#             A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
#             ToTensorV2(p=1.0)
#         ], )

In [110]:
# test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=8, shuffle=False, 
#                                                num_workers=2, 
#                                               collate_fn=collate_fn,
#                                               drop_last=True
#                                               )

# dataset_check = SteelDataset('/content/drive/MyDrive/anom_detection/task', augmentations=None)
# train_set_check, test_set_check = torch.utils.data.random_split(dataset_check, [int(len(dataset)*0.8), int(len(dataset)*0.2)])
# test_dataloader_check = torch.utils.data.DataLoader(test_set_check, batch_size=8, shuffle=True, 
#                                                num_workers=2, 
#                                                collate_fn=collate_fn
#                                                )

In [32]:
model.eval()
cpu_device = torch.device("cpu")
model.to(cpu_device)
sample = images[0].cpu().numpy()
sample = (transforms.ToTensor()(sample)).permute(1,2,0).unsqueeze(0)
outputs = model(sample)

In [33]:
outputs

[{'boxes': tensor([[ 10.2645, 105.6953,  35.7524, 197.9540],
          [  5.2397,   8.1229,  25.6115,  80.1685],
          [ 88.3357,  87.6035, 116.9236, 206.3392],
          [136.1001,  83.0471, 159.4272, 143.8305],
          [157.8567,   5.2171, 178.2957,  49.8456],
          [148.9407,   3.9425, 181.0681, 117.1028],
          [154.4190,  52.9037, 179.4691,  93.0000],
          [133.1823,  38.9374, 177.7421, 145.5850],
          [133.8241,  74.2984, 165.7640, 173.6457],
          [  4.8298,  23.8083,  33.7125, 198.6281]], grad_fn=<StackBackward>),
  'labels': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
  'scores': tensor([0.9762, 0.9632, 0.9422, 0.8786, 0.6044, 0.3774, 0.2202, 0.1339, 0.1252,
          0.0573], grad_fn=<IndexBackward>)}]

In [34]:
outputs[0]

{'boxes': tensor([[ 10.2645, 105.6953,  35.7524, 197.9540],
         [  5.2397,   8.1229,  25.6115,  80.1685],
         [ 88.3357,  87.6035, 116.9236, 206.3392],
         [136.1001,  83.0471, 159.4272, 143.8305],
         [157.8567,   5.2171, 178.2957,  49.8456],
         [148.9407,   3.9425, 181.0681, 117.1028],
         [154.4190,  52.9037, 179.4691,  93.0000],
         [133.1823,  38.9374, 177.7421, 145.5850],
         [133.8241,  74.2984, 165.7640, 173.6457],
         [  4.8298,  23.8083,  33.7125, 198.6281]], grad_fn=<StackBackward>),
 'labels': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'scores': tensor([0.9762, 0.9632, 0.9422, 0.8786, 0.6044, 0.3774, 0.2202, 0.1339, 0.1252,
         0.0573], grad_fn=<IndexBackward>)}

In [35]:
# from ensemble_boxes import *
# iou_thr = 0.5
# skip_box_thr = 0.0001
# sigma = 0.1
# for i in range(8):
#     boxes = [output['boxes'].data.cpu().numpy()/(224-1) for output in outputs]
#     scores = [output['scores'].data.cpu().numpy() for output in outputs]
#     labels = [np.ones(output['scores'].shape[0]) for output in outputs]
#     boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr)

In [36]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from ensemble_boxes import *

In [ ]:
iou_thr = 0.1
skip_box_thr = 0.4
# sigma = 0.1

In [ ]:
model.eval()
cpu_device = torch.device("cpu")
model.to(cpu_device)
batch_size = 8

for counter_to_stop, (images, targets) in enumerate(test_dataloader):
    counter = counter_to_stop * batch_size
    # print(counter)
    if counter < 20:
      for len_sample in range(batch_size):
          boxes_true = targets[len_sample]['boxes'].cpu()
          # print(boxes_true)
          sample = images[len_sample].cpu().numpy()
          sample = (transforms.ToTensor()(sample)).permute(1,2,0).unsqueeze(0)
          outputs = model(sample)
          outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
          boxes = [output['boxes'].data.cpu().numpy()/(224-1) for output in outputs]
          scores = [output['scores'].data.cpu().numpy() for output in outputs]
          labels = [np.ones(output['scores'].shape[0]) for output in outputs]
          boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
          boxes = boxes*(224-1)

          fig, ax = plt.subplots(1, 1, figsize=(16, 8))

          # for box in boxes:
          #     cv2.rectangle(sample,
          #                   (box[0], box[1]),
          #                   (box[2], box[3]),
          #                   (220, 0, 0), 3)

          for box_ind in range(len(boxes)):
              x_min,y_min,x_max,y_max = boxes[box_ind]
              rect_pred = patches.Rectangle((x_min,y_min), (x_max-x_min),(y_max-y_min),  linewidth=1, edgecolor='r', facecolor='none' )
              ax.add_patch(rect_pred)

          for box_ind_true in range(len(boxes_true)):
              x_min,y_min,x_max,y_max = boxes_true[box_ind_true]
              rect_pred = patches.Rectangle((x_min,y_min), (x_max-x_min),(y_max-y_min),  linewidth=1, edgecolor='b', facecolor='none' )
              ax.add_patch(rect_pred)
              
          ax.set_axis_off()
          # print(images[len_sample].shape)
          ax.imshow(images[len_sample].permute(1,2,0))
      

In [ ]:
# model.eval()
# cpu_device = torch.device("cpu")
# model.to(cpu_device)
# batch_size = 8

# for counter_to_stop, (images, targets) in enumerate(test_dataloader):
#     counter = counter_to_stop * batch_size
#     print(counter)
#     if counter < 20:
#       for len_sample in range(batch_size):
#           boxes = targets[len_sample]['boxes'].cpu()
#           sample = images[len_sample].cpu().numpy()
#           sample = (transforms.ToTensor()(sample)).permute(1,2,0).unsqueeze(0)
#           outputs = model(sample)
#           outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]



#           fig, ax = plt.subplots(1, 1, figsize=(16, 8))

#           # for box in boxes:
#           #     cv2.rectangle(sample,
#           #                   (box[0], box[1]),
#           #                   (box[2], box[3]),
#           #                   (220, 0, 0), 3)

#           for box_ind in range(len(outputs[0]['boxes'])):
#               x_min,y_min,x_max,y_max = outputs[0]['boxes'][box_ind]
#               rect = patches.Rectangle((x_min,y_min), (x_max-x_min),(y_max-y_min),  linewidth=1, edgecolor='r', facecolor='none' )
#               ax.add_patch(rect)
              
#           ax.set_axis_off()
#           ax.imshow(images[len_sample].permute(1,2,0))
      

In [71]:
# images, targets = next(iter(test_dataloader))

In [ ]:
# boxes = targets[5]['boxes'].cpu()
# sample = images[5].cpu().numpy()

In [ ]:
# model.eval()
# cpu_device = torch.device("cpu")
# model.to(cpu_device)

# sample = (transforms.ToTensor()(sample)).permute(1,2,0).unsqueeze(0)
# outputs = model(sample)
# outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]

In [ ]:
# outputs[0]['boxes']

In [ ]:
# sample.shape

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

fig, ax = plt.subplots(1, 1, figsize=(16, 8))

# for box in boxes:
#     cv2.rectangle(sample,
#                   (box[0], box[1]),
#                   (box[2], box[3]),
#                   (220, 0, 0), 3)

for box_ind in range(len(outputs[0]['boxes'])):
    x_min,y_min,x_max,y_max = outputs[0]['boxes'][box_ind]
    rect = patches.Rectangle((x_min,y_min), (x_max-x_min),(y_max-y_min),  linewidth=1, edgecolor='r', facecolor='none' )
    ax.add_patch(rect)
    
ax.set_axis_off()
ax.imshow(images[5].permute(1,2,0))

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches

# dataset_check = SteelDataset('/content/drive/MyDrive/anom_detection/task', augmentations=None)
# train_set_check, test_set_check = torch.utils.data.random_split(dataset_check, [int(len(dataset)*0.8), int(len(dataset)*0.2)])
# train_dataloader_check = torch.utils.data.DataLoader(train_set_check, batch_size=16, shuffle=True, 
#                                                num_workers=2, 
#                                                collate_fn=collate_fn
#                                                )

# train_features, train_labels = next(iter(train_dataloader_check))

# for sample in range(len(train_features)):
#   img = transforms.functional.pil_to_tensor(train_features[sample]).permute(1,2,0)
#   boxes = train_labels[sample]

#   fig, ax = plt.subplots()
#   ax.imshow(img)

#   # Create a bboxes
#   for box_ind in range(len(boxes)):
#     x_min,y_min,x_max,y_max = boxes[box_ind]
#     rect = patches.Rectangle((x_min,y_min), (x_max-x_min),(y_max-y_min),  linewidth=1, edgecolor='r', facecolor='none' )
#     ax.add_patch(rect)

# plt.show()